# Shutting Rogue DHCP Servers

In [ ]:
import exsh
import re

vlan_output = exsh.clicmd("show vlan", capture=True)
vlan_list = []
for line in vlan_output.split("\n"):
    match = re.search(r"(\S+).*\s\d+", line)  # Extract VLAN name
    if match:
        vlan_list.append(match.group(1))

for vlan in vlan_list:
    print(f"Checking VLAN {vlan} for rogue DHCP servers...")
    dhcp_output = exsh.clicmd(f"show ip-security dhcp-snooping violations vlan {vlan}", capture=True)
    rogue_servers = []
    for line in dhcp_output.split("\n"):
        match = re.search(r"(\S+)\s+(\S+)\s+Rogue", line)  # Extract IP and port
        if match:
            ip, port = match.groups()
            rogue_servers.append((ip, port))

    if rogue_servers:
        for ip, port in rogue_servers:
            exsh.clicmd(f"disable port {port}")
            print(f"Rogue DHCP server detected on {port} (IP: {ip}) - Port Disabled!")
    else:
        print(f"No rogue DHCP servers detected on VLAN {vlan}.")